Using firmware from
    https://micropython.org/download/ESP32_GENERIC/


In [251]:
%serialconnect

Found serial ports: /dev/ttyUSB0, /dev/ttyS0 
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 1, in <module>
ImportError: no module named 'mqtt_as'
MicroPython v1.22.2 on 2024-02-22; Generic ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [256]:
%sendtofile --source /home/julian/repositories/micropython-mqtt/mqtt_as/mqtt_as.py

Sent 800 lines (29331 bytes) to mqtt_as.py.


In [242]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
boardname    esp32arcade
mqttbroker   mosquitto.doesliverpool.xyz

Sent 4 lines (116 bytes) to config.txt.


In [271]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Found serial ports: /dev/ttyUSB1, /dev/ttyS0 
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [266]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 88 lines (2992 bytes) to stdmqttas.py.
Sent 39 lines (1151 bytes) to utils.py.


In [272]:
%sendtofile main.py

from mqtt_as import MQTTClient
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask, flashledconnectedtask
import uasyncio as asyncio
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count
import time

from arcadeboards import setupallarcadeboards, setbuttonlight, newbuttondown
from arcadeboards import row1, row2, brow1, brow2

topicmessage = topicstem+"/message"
topiccmd = topicstem+"/cmd"
topicstatus = topicstem+"/status"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"

topicrow1 = "spscontroller/controller/video"
topicrow2 = "spscontroller/controller/audio"

row1mess = [ "Videos/black.mp4", "Videos/white.mp4", "Videos/red.mp4", "Videos/yellow.mp4", "Videos/blue.mp4", "Videos/green.mp4" ]
row2mess = [ "Music/silent.mp3", "Music/nostalgia.mp3", "Music/joyful.mp3", "Music/silly.mp3", "Music/romantic.mp3", "Music/tense.mp3" ] 

async def arcadetask(client):
    setupallarcadeboards()
    while not client.isconnected():
        setbuttonlight(row1[0], 50000)
        await asyncio.sleep_ms(200)
        setbuttonlight(row1[0], 5000)
        await asyncio.sleep_ms(600)

    currentrow1button = row1[0]
    currentrow2button = row2[0]
    while True:
        k = newbuttondown(50)
        if k:
            if k in brow1:
                i1 = brow1.index(k)
                if currentrow1button:
                    setbuttonlight(currentrow1button, 5000)
                currentrow1button = row1[i1]
                setbuttonlight(currentrow1button, 50000)
                await client.publish(topicrow1, row1mess[i1])
            if k in brow2:
                i2 = brow2.index(k)
                if currentrow2button:
                    setbuttonlight(currentrow2button, 5000)
                currentrow2button = row2[i2]
                setbuttonlight(currentrow2button, 50000)
                await client.publish(topicrow2, row2mess[i2])
        await asyncio.sleep_ms(50)
        
def callbackcmd(topic, msg, retained):
    print("callbackcmd", topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd)
    await client.subscribe(topicmessage)
    await client.publish(topicip, ipnumber, retain=True, qos=1)
    client.connectioncount += 1
    print("subscribed")
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]

print(config)
client = MQTTClient(config)
client.DEBUG = True
client.connectioncount = 0

aloop = asyncio.get_event_loop()
aloop.create_task(arcadetask(client))
aloop.create_task(mqttconnecttask(client, None))
aloop.run_forever()


Sent 80 lines (2932 bytes) to main.py.
